# Tool Sequence Analysis

This notebook analyzes the sequence of tool calls in RCA tasks. It ingests experiment results, extracts tool usage patterns, and visualizes the flow of tool calls using a Sankey diagram.

**Note**: To export static images (PNG), you need the `kaleido` package installed (`pip install kaleido`).

In [ ]:
import os
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from dotenv import load_dotenv

# Load environment variables
root_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
load_dotenv(os.path.join(root_dir, '.env'))

RESULTS_DIR = os.environ.get("RESULTS_PATH")
print(f"Results Directory: {RESULTS_DIR}")

# Output directory for static plots
PLOTS_DIR = "analysis_plots"
os.makedirs(PLOTS_DIR, exist_ok=True)
print(f"Plots will be saved to: {PLOTS_DIR}")

## Select Experiments

In [ ]:
if RESULTS_DIR and os.path.exists(RESULTS_DIR):
    experiment_batches = [d for d in os.listdir(RESULTS_DIR) if os.path.isdir(os.path.join(RESULTS_DIR, d))]
    experiment_batches.sort()
    
    print("Available Experiment Batches:")
    for i, d in enumerate(experiment_batches, 1):
        dir_path = os.path.join(RESULTS_DIR, d)
        json_count = sum(1 for f in os.listdir(dir_path) if f.endswith('.json'))
        print(f"{i}) {d} ({json_count} results)")
else:
    print("Results directory not found or empty.")
    experiment_batches = []

In [ ]:
# Interactive Selection (Default to last if no input mechanism in non-interactive run)
selected_indices = input("Enter experiment batch IDs (comma-separated, e.g., 1,2): ")

selected_dirs = []
try:
    if selected_indices.strip():
        indices = [int(x.strip()) - 1 for x in selected_indices.split(',')]
        for idx in indices:
            if 0 <= idx < len(experiment_batches):
                selected_dirs.append(os.path.join(RESULTS_DIR, experiment_batches[idx]))
            else:
                print(f"Warning: Index {idx+1} out of range.")
    else:
        print("No input provided.")
except ValueError:
    print("Invalid input.")

if not selected_dirs and experiment_batches:
    print("Defaulting to the most recent batch.")
    selected_dirs = [os.path.join(RESULTS_DIR, experiment_batches[-1])]
    
print(f"Selected {len(selected_dirs)} folders for analysis.")

## Data Ingestion & Processing

In [ ]:
def extract_tool_calls(message_history):
    """Extracts a sequential list of tool names from message history."""
    tool_sequence = []
    if not isinstance(message_history, list):
        return tool_sequence
        
    for message in message_history:
        # Check if message is an AI message with tool calls
        if isinstance(message, dict) and message.get("type") == "AIMessage":
            tool_calls = message.get("tool_calls", [])
            for tc in tool_calls:
                if isinstance(tc, dict) and "name" in tc:
                    tool_sequence.append(tc["name"])
    return tool_sequence

records = []

for folder in selected_dirs:
    if not os.path.exists(folder):
        continue
        
    files = [f for f in os.listdir(folder) if f.endswith('.json')]
    print(f"Processing {folder}... Found {len(files)} JSON files.")
    
    for file_name in files:
        file_path = os.path.join(folder, file_name)
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
            
            agent_id = data.get("agent_id", "Unknown")
            app_name = data.get("app_name") or data.get("testbed", {}).get("application_name", ["Unknown"])[0]
            fault_type = data.get("testbed", {}).get("fault_name", "Unknown")
            
            # Evaluation Data
            evaluation = data.get("evaluation", {})
            eval_detection = evaluation.get("detection")
            eval_localization = evaluation.get("localization")
            eval_rca_score = evaluation.get("rca_score")

            # Experiment Stats
            stats = data.get("stats", {})
            execution_time = stats.get("execution_time_seconds")
            total_tokens = stats.get("total_tokens")

            # Process RCA Analyses
            rca_list = data.get("rca_analyses_list", [])
            if not rca_list:
                continue # Skip if no analysis
                
            for i, analysis in enumerate(rca_list, 1):
                message_history = analysis.get("message_history", [])
                tool_sequence = extract_tool_calls(message_history)
                
                # Create record
                row = {
                    "Agent ID": agent_id,
                    "Application Name": app_name,
                    "Fault Type": fault_type,
                    "Eval Detection": eval_detection,
                    "Eval Localization": eval_localization,
                    "Eval RCA Score": eval_rca_score,
                    "Execution Time (s)": execution_time,
                    "Total Tokens": total_tokens,
                    "RCA Task Index": f"RCA Task {i}",
                    "Total Tools": len(tool_sequence)
                }
                
                # Add Tool 1 to Tool 10
                for t_idx in range(10):
                    col_name = f"Tool {t_idx + 1}"
                    if t_idx < len(tool_sequence):
                        row[col_name] = tool_sequence[t_idx]
                    else:
                        row[col_name] = None # Or "End"
                
                records.append(row)
                
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

df = pd.DataFrame(records)
print(f"Created DataFrame with {len(df)} rows.")
df.head()

## Visualization: Tool Sequence Sankey Diagram

In [ ]:
# Prepare data for Sankey
# We strictly want sequences: Tool 1 -> Tool 2 -> Tool 3 ...

# Sources: Index of the source node
# Targets: Index of the target node
# Values: Count of transitions

# To make the Sankey readable (layers), we need unique node names per step.
# E.g., "Step1: get_logs" is different from "Step2: get_logs".

max_depth = 10
links = []

# Initialize node mapping
# Format: "{StepIndex}_{ToolName}" -> ID
node_label_map = {}
node_labels = []
node_counter = 0

def get_node_id(step, tool_name):
    global node_counter
    label = f"Step {step}: {tool_name}"
    if label not in node_label_map:
        node_label_map[label] = node_counter
        node_labels.append(tool_name) # Just show tool name in plot, but ID distinguishes step
        node_counter += 1
    return node_label_map[label]

source_indices = []
target_indices = []
values = []

# Aggregate transitions
transitions = {}

for _, row in df.iterrows():
    # Walk through tools 1 to 10
    for i in range(1, max_depth):
        current_col = f"Tool {i}"
        next_col = f"Tool {i+1}"
        
        current_tool = row[current_col]
        next_tool = row[next_col]
        
        # Validate sequence
        if pd.isna(current_tool):
            break # End of sequence
        
        if pd.isna(next_tool):
            # Transition to "End" node for this step
            next_tool_label = "End"
        else:
            next_tool_label = next_tool
            
        # Key: (Step_i, ToolA, Step_i+1, ToolB)
        key = (i, current_tool, i+1, next_tool_label)
        transitions[key] = transitions.get(key, 0) + 1
        
        if next_tool_label == "End":
            break

# Build Sankey Arrays
for (step_from, tool_from, step_to, tool_to), count in transitions.items():
    src_id = get_node_id(step_from, tool_from)
    tgt_id = get_node_id(step_to, tool_to)
    
    source_indices.append(src_id)
    target_indices.append(tgt_id)
    values.append(count)

# Create Plot
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = node_labels,
      color = "blue"
    ),
    link = dict(
      source = source_indices,
      target = target_indices,
      value = values
  ))])

fig.update_layout(title_text="Tool Call Sequence Flow (Steps 1-10)", font_size=10)
fig.show()

# Save Static Image
try:
    static_file = os.path.join(PLOTS_DIR, "tool_sequence_sankey.png")
    fig.write_image(static_file)
    print(f"Saved static Sankey diagram to {static_file}")
except ValueError as e:
    print(f"Could not save static image. Ensure 'kaleido' is installed. Error: {e}")

## Tool Distribution Analysis

Analyzes the frequency of tools used at each specific step.

In [ ]:
# Distribution of First Tool
first_tool_counts = df['Tool 1'].value_counts()
fig = px.bar(first_tool_counts, title="Distribution of First Tool Called")
fig.show()

# Save Static Image
try:
    static_file = os.path.join(PLOTS_DIR, "first_tool_distribution.png")
    fig.write_image(static_file)
    print(f"Saved static distribution plot to {static_file}")
except ValueError as e:
    print(f"Could not save static image. Ensure 'kaleido' is installed. Error: {e}")